In [1]:
import os
print(os.getcwd())

/Users/sazzad14/Documents/Stanford/CS-231N/portalcut


In [2]:
import os
import pandas as pd

def parse_kitti_labels(label_dir):
    annotations = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file)) as f:
                for line in f.readlines():
                    elements = line.strip().split()
                    annotations.append({
                        'filename': label_file.replace('.txt', '.png'),
                        'class': elements[0],
                        'bbox': [float(elements[4]), float(elements[5]), float(elements[6]), float(elements[7])]
                    })
    return pd.DataFrame(annotations)

label_dir = 'datasets/KITTI/training/label_2'
annotations = parse_kitti_labels(label_dir)
annotations.to_csv('kitti_annotations.csv', index=False)


In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('kitti_annotations.csv')
df

,filename,class,bbox
0,006145.png,Car,"[595.09, 177.15, 637.82, 217.56]"
1,006145.png,Car,"[0.0, 185.06, 198.62, 277.08]"
2,006145.png,Car,"[812.56, 169.04, 1023.51, 293.58]"
3,006145.png,Car,"[764.98, 172.7, 947.97, 261.04]"
4,006145.png,Car,"[708.54, 177.97, 852.27, 243.88]"
...,...,...,...
51860,001633.png,Car,"[0.0, 181.72, 246.84, 274.75]"
51861,001633.png,Car,"[78.07, 181.92, 288.68, 255.62]"
51862,001633.png,Car,"[149.73, 170.29, 321.46, 242.46]"
51863,001633.png,Car,"[488.45, 179.28, 540.52, 217.39]"


In [6]:
np.random.seed(42)
shuffled_indices = np.random.permutation(len(df))
test_set_size = int(len(df) * 0.2)
test_indices = shuffled_indices[:test_set_size]
train_indices = shuffled_indices[test_set_size:]

train_df = df.iloc[train_indices]
val_df = df.iloc[test_indices]


In [7]:
train_df.to_csv('train_annotations.csv', index=False)
val_df.to_csv('val_annotations.csv', index=False)


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2

class KITTIDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes = self.img_labels.iloc[idx, 2:6].astype('float').values.reshape(-1, 4)
        labels = 10

        target = {'boxes': torch.tensor(boxes, dtype=torch.float32), 'labels': labels}

        if self.transform:
            image = self.transform(image=image)['image']
        
        return image, target

train_dataset = KITTIDataset('train_annotations.csv', 'KITTI/training/image_2')
val_dataset = KITTIDataset('val_annotations.csv', 'KITTI/training/image_2')


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))


41492
41492


In [14]:
import pandas as pd

# Assuming the CSV file is in the same directory as your script
df = pd.read_csv('kitti_annotations.csv', header=None)

# The second column contains the classes
classes = df[1]

# Find the unique classes
distinct_classes = classes.unique()

# Print the number of distinct classes
print(len(distinct_classes))

10


In [11]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model(num_classes=10) 


/Users/sazzad14/Documents/Stanford/CS-231N/portalcut/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sazzad14/Documents/Stanford/CS-231N/portalcut/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /Users/sazzad14/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:06<00:00, 24.7MB/s] 
